In [5]:
import numpy as np
import pandas as pd
from scipy.stats import norm, chi2

# import the table, rename columns
col=['I', 'Q', 'C', 'D']
df=pd.read_table('invest.txt', header=None, names=col)

# create a column for constant 1
df['one']=np.ones(df.shape[0])
cols=np.append([co for co in col[1:]], 'one')

# calculate OLS estimator
X=df[cols]
y=df['I']
beta=np.dot(
    np.linalg.inv(np.dot(X.T, X)), 
    np.dot(X.T, y))

# get residual and its square
def get_e_hat(row):
    return row['I']-np.dot(row[cols].T, beta)
df['res']=df.apply(get_e_hat, axis=1)
df['e_hat_sqr']=df['res']**2

# calculate std_error by white
e_e=np.diag(df['e_hat_sqr'])
left=np.linalg.inv(np.dot(X.T, X))
mid=np.dot(np.dot(X.T, e_e), X)
right=left
White=np.dot(np.dot(left, mid), right)
std_err_white=[np.sqrt(White[i][i]) for i in range(X.shape[1])]

In [6]:
# a)
print('beta:',beta, '\n'
      'std_err:',std_err_white)

beta: [-0.42128515  7.22930658  0.73638279  7.50992276] 
std_err: [0.28009623653862048, 1.6512910031464012, 0.38598096897689016, 0.39533874430485133]


In [176]:
# b)
# construct asymptotic interval for alpha=0.05
c=norm.ppf(0.975)
asym_con_int=[[beta[i]-c*std_err_white[i], beta[i]+c*std_err_white[i]] for i in range(X.shape[1])]
asym_con_int

[[-0.97026368590078249, 0.12769338574103362],
 [3.992835681587811, 10.465777469911737],
 [-0.020126011225698459, 1.4928915845994548],
 [6.735073058227262, 8.2847724592888579]]

In [178]:
# c1)
# construct the Wald statistics for the joint hypo
R=np.array([[0, 1, 0, 0], [0, 0, 1, 0]]).T
left=np.dot(R.T, beta)
mid=np.linalg.inv(np.dot(np.dot(R.T, White), R))
right=left
Wald=np.dot(np.dot(left, mid), right)
# get critical value and 
c=chi2.ppf(0.95, df=np.linalg.matrix_rank(R))
print('Wald:', Wald, '\n',
      'crit_value', c, '\n',
['Fail to Reject','Reject'][Wald>c], '\n',
      'p-value:', 1-chi2.cdf(Wald, 2))

Wald: 24.1344723483 
 crit_value 5.99146454711 
 Reject 
 p-value: 5.74468100345e-06


In [179]:
# c2)
# construct the T-statistics for the sinple hypo
T=beta[0]/std_err_white[0]
c=norm.ppf(0.975)
print('T:', T, '\n',
      'crit_value', c, '\n',
['Fail to Reject','Reject'][T>c], '\n',
      'p-value:', 1-norm.cdf(T))

T: -1.50407286898 
 crit_value 1.95996398454 
 Fail to Reject 
 p-value: 0.933718697399


In [180]:
# d)
# add new variables
df['Q2']=df['Q']**2
df['C2']=df['C']**2
df['D2']=df['D']**2
df['QC']=df['Q']*df['C']
df['QD']=df['Q']*df['D']
df['CD']=df['C']*df['D']

# regression on all of them
X=df[['Q', 'C', 'D', 'Q2', 'C2', 'D2', 'QC', 'QD', 'CD','one']]
y=df['I']
beta=np.dot(
    np.linalg.inv(np.dot(X.T, X)), 
    np.dot(X.T, y))

# get residual and its square
def get_e_hat_new(row):
    return row['I']-np.dot(np.transpose(row[['Q', 'C', 'D', 'Q2', 'C2', 'D2', 'QC', 'QD', 'CD','one']]), beta)
df['res_new']=df.apply(get_e_hat_new, axis=1)
df['e_hat_sqr_new']=df['res_new']**2

# calculate std_error by white
e_e=np.diag(df['e_hat_sqr_new'])
left=np.linalg.inv(np.dot(X.T, X))
mid=np.dot(np.dot(X.T, e_e), X)
right=left
White=np.dot(np.dot(left, mid), right)
std_err_white=[np.sqrt(White[i][i]) for i in range(X.shape[1])]

In [181]:
# construct the Wald statistics for the joint hypo
R=np.concatenate((np.zeros((6, 3)),np.identity(6), np.zeros((6, 1))), axis=1).T
left=np.dot(R.T, beta)
mid=np.linalg.inv(np.dot(np.dot(R.T, White), R))
right=left
Wald=np.dot(np.dot(left, mid), right)
# get critical value and 
c=chi2.ppf(0.95, df=np.linalg.matrix_rank(R))
print('Wald:', Wald, '\n',
      'crit_value', c, '\n',
['Fail to Reject','Reject'][Wald>c], '\n',
      'p-value:', 1-chi2.cdf(Wald, 6))

Wald: 51.4963142371 
 crit_value 12.5915872437 
 Reject 
 p-value: 2.35440822394e-09
